# 🎨 Trellis NIM - Simple Deployment & Inference

A streamlined notebook for deploying NVIDIA Trellis NIM and generating 3D models from text prompts using command line interface.

## What This Notebook Does:
1. **🔧 GPU Check** - Verify L40s GPU availability  
2. **🚀 Deploy NIM** - Run the Trellis NIM container
3. **⚡ Generate Models** - Command line 3D generation from text prompts

## Key Features:
- ✅ **Simple setup** - Just run the cells in order
- ✅ **Command line generation** - Direct API calls to NIM service
- ✅ **GLB output** - High-quality 3D models ready for Blender, Unity, etc.
- ✅ **Container management** - Easy start, stop, and status checking

**🚀 From setup to 3D models in minutes!**

---


## Step 1: Check GPU Availability

Verify that we have access to an L40s GPU:


In [ ]:
!nvidia-smi


## Step 2: Deploy Trellis NIM Container

**⚠️ IMPORTANT**: Replace `<PASTE_API_KEY_HERE>` with your actual NGC API key from https://ngc.nvidia.com/setup/api-key


In [ ]:
# Set your NGC API key here
import os
os.environ['NGC_API_KEY'] = '<PASTE_API_KEY_HERE>'  # Replace with your actual API key

# Verify the API key is set
if os.environ['NGC_API_KEY'] == '<PASTE_API_KEY_HERE>':
    print('❌ Please replace <PASTE_API_KEY_HERE> with your actual NGC API key!')
else:
    print('✅ NGC API key is set')


In [ ]:
# Docker login to NGC registry
!echo "$NGC_API_KEY" | docker login nvcr.io --username '$oauthtoken' --password-stdin


In [ ]:
# Create cache directory
!mkdir -p ~/.cache/nim
!chmod 777 ~/.cache/nim
!echo "Cache directory created at: ~/.cache/nim"


In [ ]:
# Start Trellis NIM container (this will run in the background)
# Note: This may take several minutes to download and start the first time

!docker run -d --name=nim-server \
  --runtime=nvidia --gpus='"device=0"' \
  -e NGC_API_KEY=$NGC_API_KEY \
  -p 8000:8000 \
  -v ~/.cache/nim:/opt/nim/.cache/ \
  nvcr.io/nim/microsoft/trellis:latest

print('🚀 Trellis NIM container started!')
print('⏳ Please wait 2-5 minutes for the container to fully initialize...')


In [ ]:
# Check container status
!docker ps | grep nim-server
!echo "\n📊 Container logs (last 10 lines):"
!docker logs nim-server --tail 10


## Step 3: Generate 3D Models (Command Line Method)

**🌐 RECOMMENDED: Use the Web Interface (Cell 14) for the best experience!**

Alternatively, you can generate models via command line by changing the prompt below and running the cell:


In [ ]:
#CHANGE THIS PROMPT TO GENERATE DIFFERENT 3D MODELS. BRANDS OR SPECIFIC PRODUCTS WILL FAIL THE GENERATION. 
PROMPT = "A simple coffee shop interior"

#Change output filename
OUTPUT_FILE = "result.glb"

# Optional: Change the seed for different variations (0 = random)
SEED = 0

# First, let's test if the NIM container is responding
print("🔍 Testing NIM container connectivity...")
health_check = !curl -s http://localhost:8000/health || curl -s http://localhost:8000/ || echo "CONNECTION_FAILED"

if "CONNECTION_FAILED" in ' '.join(health_check):
    print("❌ Cannot connect to NIM container at http://localhost:8000")
    print("💡 Make sure the container is running with: docker ps | grep nim-server")
    print("💡 Check container logs with: docker logs nim-server")
else:
    print("✅ NIM container is responding")

print("\n📡 Sending generation request to Trellis NIM...")

# Use the exact curl command format from your original request
import subprocess
import json
import base64

# Create the JSON payload
json_payload = json.dumps({
    "prompt": PROMPT,
    "seed": SEED
})

# Execute curl command exactly as specified
curl_cmd = [
    'curl', '-X', 'POST', 'http://localhost:8000/v1/infer',
    '-H', 'Accept: application/json',
    '-H', 'Content-Type: application/json',
    '-d', json_payload,
    '--silent'
]

print(f"🎯 Generating: '{PROMPT}'")
print("⏳ This may take 1-3 minutes...")

try:
    # Run the curl command
    result = subprocess.run(curl_cmd, capture_output=True, text=True, timeout=300)
    
    if result.returncode != 0:
        print(f"❌ Curl command failed with return code: {result.returncode}")
        print(f"Error: {result.stderr}")
        print(f"Command: {' '.join(curl_cmd)}")
    else:
        response_text = result.stdout.strip()
        print(f"📨 Raw response length: {len(response_text)} characters")
        print(f"📨 Response preview: {response_text[:300]}...")
        
        if not response_text:
            print("❌ Empty response from server")
        elif response_text.startswith('<!DOCTYPE') or '<html' in response_text:
            print("❌ Received HTML response instead of JSON - check if container is fully started")
        else:
            try:
                # Parse JSON response
                response_data = json.loads(response_text)
                print("✅ Valid JSON response received")
                
                # Check for artifacts in response
                if 'artifacts' in response_data and len(response_data['artifacts']) > 0:
                    artifact = response_data['artifacts'][0]
                    if 'base64' in artifact:
                        base64_data = artifact['base64']
                        
                        # Decode and save the GLB file
                        glb_data = base64.b64decode(base64_data)
                        
                        with open(OUTPUT_FILE, 'wb') as f:
                            f.write(glb_data)
                        
                        print(f"✅ SUCCESS! 3D model saved as: {OUTPUT_FILE}")
                        print(f"📊 File size: {len(glb_data) / 1024:.2f} KB")
                        print(f"🎯 You can now open {OUTPUT_FILE} in any 3D viewer or Blender!")
                    else:
                        print("❌ No base64 data in artifact")
                        print(f"Artifact keys: {list(artifact.keys())}")
                else:
                    print("❌ No artifacts found in response")
                    print(f"Response keys: {list(response_data.keys())}")
                    if 'error' in response_data:
                        print(f"Error from server: {response_data['error']}")
                        
            except json.JSONDecodeError as e:
                print(f"❌ Failed to parse JSON response: {e}")
                print(f"Raw response: {response_text}")

except subprocess.TimeoutExpired:
    print("❌ Request timed out (>5 minutes)")
except Exception as e:
    print(f"❌ Unexpected error: {e}")
    import traceback
    traceback.print_exc()


## 🎉 That's it!

### To generate more 3D models:
1. Go back to the cell above with `PROMPT = "..."` and `OUTPUT_FILE = "... .glb"`
2. Change the prompt to whatever you want
3. Run both cells again

### Example prompts to try:
- `"A modern chair"`
- `"A futuristic car"`
- `"A wooden table"`
- `"A cozy bedroom"`
- `"A space station interior"`

### Useful commands:


In [ ]:
# Check if NIM container is running
!docker ps | grep nim-server


## Step 3: Generate 3D Models

Now you can generate 3D models by changing the prompt below and running the cell:


In [ ]:
# List generated GLB files
!ls -la *.glb






<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Trellis 3D Generator</title>
    <script type="module" src="https://unpkg.com/@google/model-viewer/dist/model-viewer.min.js"></script>
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        
        body {
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            display: flex;
        }
        
        .container {
            display: flex;
            width: 100%;
            max-width: 1400px;
            margin: 20px auto;
            background: white;
            border-radius: 20px;
            box-shadow: 0 20px 40px rgba(0, 0, 0, 0.1);
            overflow: hidden;
        }
        
        .left-panel {
            flex: 1;
            padding: 40px;
            background: #f8fafc;
        }
        
        .right-panel {
            flex: 1.5;
            background: #1a1a2e;
            position: relative;
            display: flex;
            flex-direction: column;
        }
        
        h1 {
            color: #2d3748;
            font-size: 2.5rem;
            font-weight: 700;
            margin-bottom: 10px;
        }
        
        .subtitle {
            color: #718096;
            font-size: 1.1rem;
            margin-bottom: 30px;
        }
        
        .form-group {
            margin-bottom: 25px;
        }
        
        label {
            display: block;
            font-weight: 600;
            color: #4a5568;
            margin-bottom: 8px;
            font-size: 0.95rem;
        }
        
        #promptInput {
            width: 100%;
            padding: 15px 20px;
            font-size: 16px;
            border: 2px solid #e2e8f0;
            border-radius: 12px;
            background: white;
            transition: all 0.3s ease;
            font-family: inherit;
        }
        
        #promptInput:focus {
            outline: none;
            border-color: #667eea;
            box-shadow: 0 0 0 3px rgba(102, 126, 234, 0.1);
        }
        
        .controls {
            display: flex;
            gap: 15px;
            margin-bottom: 25px;
        }
        
        .seed-input {
            flex: 1;
        }
        
        #seedInput {
            width: 100%;
            padding: 12px 15px;
            border: 2px solid #e2e8f0;
            border-radius: 8px;
            font-size: 14px;
        }
        
        #generateBtn {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            border: none;
            padding: 15px 30px;
            font-size: 16px;
            font-weight: 600;
            border-radius: 12px;
            cursor: pointer;
            transition: all 0.3s ease;
            box-shadow: 0 4px 15px rgba(102, 126, 234, 0.3);
        }
        
        #generateBtn:hover:not(:disabled) {
            transform: translateY(-2px);
            box-shadow: 0 6px 20px rgba(102, 126, 234, 0.4);
        }
        
        #generateBtn:disabled {
            opacity: 0.6;
            cursor: not-allowed;
            transform: none;
        }
        
        .status {
            padding: 15px 20px;
            border-radius: 8px;
            margin-bottom: 20px;
            font-weight: 500;
        }
        
        .status.info {
            background: #ebf8ff;
            color: #2b6cb0;
            border: 1px solid #90cdf4;
        }
        
        .status.success {
            background: #f0fff4;
            color: #276749;
            border: 1px solid #9ae6b4;
        }
        
        .status.error {
            background: #fed7d7;
            color: #c53030;
            border: 1px solid #feb2b2;
        }
        
        .examples {
            background: white;
            padding: 20px;
            border-radius: 12px;
            border: 1px solid #e2e8f0;
        }
        
        .examples h3 {
            color: #2d3748;
            margin-bottom: 15px;
            font-size: 1.1rem;
        }
        
        .example-item {
            padding: 8px 12px;
            background: #f7fafc;
            border-radius: 6px;
            cursor: pointer;
            transition: all 0.2s ease;
            margin-bottom: 8px;
            font-size: 0.9rem;
            color: #4a5568;
        }
        
        .example-item:hover {
            background: #edf2f7;
            color: #2d3748;
        }
        
        model-viewer {
            width: 100%;
            height: 60%;
            background: linear-gradient(135deg, #1a1a2e 0%, #16213e 50%, #0f3460 100%);
        }
        
        .viewer-info {
            padding: 30px;
            color: white;
            flex: 1;
            display: flex;
            flex-direction: column;
            justify-content: center;
            align-items: center;
            text-align: center;
        }
        
        .viewer-placeholder {
            font-size: 1.2rem;
            color: #cbd5e0;
            margin-bottom: 15px;
        }
        
        .viewer-instructions {
            color: #a0aec0;
            font-size: 0.9rem;
            max-width: 300px;
            line-height: 1.6;
        }
        
        .loading-spinner {
            display: none;
            width: 40px;
            height: 40px;
            border: 4px solid #f3f3f3;
            border-top: 4px solid #667eea;
            border-radius: 50%;
            animation: spin 1s linear infinite;
            margin: 20px auto;
        }
        
        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }
        
        .hidden {
            display: none !important;
        }
        
        @media (max-width: 768px) {
            .container {
                flex-direction: column;
                margin: 10px;
            }
            .left-panel, .right-panel {
                flex: none;
            }
            .right-panel {
                min-height: 400px;
            }
            h1 {
                font-size: 2rem;
            }
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="left-panel">
            <h1>🎨 Trellis 3D</h1>
            <p class="subtitle">Generate 3D models from text prompts using NVIDIA Trellis NIM</p>
            
            <div class="form-group">
                <label for="promptInput">Enter your prompt:</label>
                <input type="text" id="promptInput" 
                       placeholder="A modern chair with sleek design..." 
                       value="A modern wooden chair">
            </div>
            
            <div class="controls">
                <div class="seed-input">
                    <label for="seedInput">Seed (0 = random):</label>
                    <input type="number" id="seedInput" value="0" min="0">
                </div>
            </div>
            
            <button id="generateBtn">🚀 Generate 3D Model</button>
            <div class="loading-spinner" id="loadingSpinner"></div>
            
            <div id="status" class="status hidden"></div>
            
            <div class="examples">
                <h3>💡 Example Prompts:</h3>
                <div class="example-item" data-prompt="A modern wooden chair">A modern wooden chair</div>
                <div class="example-item" data-prompt="A futuristic sports car">A futuristic sports car</div>
                <div class="example-item" data-prompt="A cozy living room">A cozy living room</div>
                <div class="example-item" data-prompt="A medieval castle tower">A medieval castle tower</div>
                <div class="example-item" data-prompt="A space station module">A space station module</div>
                <div class="example-item" data-prompt="A vintage coffee shop interior">A vintage coffee shop interior</div>
            </div>
        </div>
        
        <div class="right-panel">
            <model-viewer id="modelViewer" class="hidden"
                         camera-controls 
                         enable-pan
                         auto-rotate
                         shadow-intensity="1"
                         environment-image="neutral">
            </model-viewer>
            
            <div class="viewer-info" id="viewerPlaceholder">
                <div class="viewer-placeholder">🎯 3D Model Viewer</div>
                <div class="viewer-instructions">
                    Enter a prompt and click "Generate 3D Model" to see your creation appear here. 
                    You'll be able to rotate, zoom, and interact with the generated 3D model.
                </div>
            </div>
        </div>
    </div>

    <script>
        // NIM Service Integration
        class NIMService {
            constructor() {
                this.config = {
                    nim_endpoint: "http://localhost:8000",
                    infer_endpoint: "http://localhost:8000/v1/infer",
                    health_endpoint: "http://localhost:8000/health"
                };
                this.currentModelUrl = null;
            }
            
            async generateModel(prompt, seed = 0) {
                const payload = { prompt: prompt, seed: seed };
                
                try {
                    console.log('🚀 Sending request to NIM:', payload);
                    const response = await fetch(this.config.infer_endpoint, {
                        method: 'POST',
                        headers: {
                            'Accept': 'application/json',
                            'Content-Type': 'application/json'
                        },
                        body: JSON.stringify(payload)
                    });
                    
                    if (!response.ok) {
                        throw new Error(`HTTP error! status: ${response.status}`);
                    }
                    
                    const data = await response.json();
                    console.log('✅ NIM response received:', data);
                    
                    if (data.artifacts && data.artifacts.length > 0) {
                        const artifact = data.artifacts[0];
                        if (artifact.base64) {
                            return this.base64ToBlob(artifact.base64);
                        }
                    }
                    
                    throw new Error('No valid artifact in response');
                    
                } catch (error) {
                    console.error('❌ NIM generation error:', error);
                    throw error;
                }
            }
            
            base64ToBlob(base64Data) {
                try {
                    const byteCharacters = atob(base64Data);
                    const byteNumbers = new Array(byteCharacters.length);
                    for (let i = 0; i < byteCharacters.length; i++) {
                        byteNumbers[i] = byteCharacters.charCodeAt(i);
                    }
                    const byteArray = new Uint8Array(byteNumbers);
                    return new Blob([byteArray], { type: 'model/gltf-binary' });
                } catch (error) {
                    console.error('❌ Base64 decode error:', error);
                    throw new Error('Failed to decode model data');
                }
            }
            
            createModelUrl(blob) {
                if (this.currentModelUrl) {
                    URL.revokeObjectURL(this.currentModelUrl);
                }
                this.currentModelUrl = URL.createObjectURL(blob);
                return this.currentModelUrl;
            }
            
            async checkHealth() {
                try {
                    const response = await fetch(this.config.health_endpoint, { timeout: 5000 });
                    return response.ok;
                } catch (error) {
                    try {
                        const response = await fetch(this.config.nim_endpoint, { timeout: 5000 });
                        return response.ok;
                    } catch (fallbackError) {
                        return false;
                    }
                }
            }
        }

        // Main Application Logic
        class TrellisApp {
            constructor() {
                this.nimService = new NIMService();
                this.initializeElements();
                this.attachEventListeners();
                this.checkNIMStatus();
            }
            
            initializeElements() {
                this.promptInput = document.getElementById('promptInput');
                this.seedInput = document.getElementById('seedInput');
                this.generateBtn = document.getElementById('generateBtn');
                this.loadingSpinner = document.getElementById('loadingSpinner');
                this.statusDiv = document.getElementById('status');
                this.modelViewer = document.getElementById('modelViewer');
                this.viewerPlaceholder = document.getElementById('viewerPlaceholder');
            }
            
            attachEventListeners() {
                this.generateBtn.addEventListener('click', () => this.generateModel());
                
                this.promptInput.addEventListener('keypress', (e) => {
                    if (e.key === 'Enter' && !this.generateBtn.disabled) {
                        this.generateModel();
                    }
                });
                
                document.querySelectorAll('.example-item').forEach(item => {
                    item.addEventListener('click', () => {
                        this.promptInput.value = item.dataset.prompt;
                        this.promptInput.focus();
                    });
                });
            }
            
            async checkNIMStatus() {
                try {
                    const isHealthy = await this.nimService.checkHealth();
                    if (isHealthy) {
                        this.showStatus('✅ Connected to Trellis NIM service', 'success');
                    } else {
                        this.showStatus('⚠️ NIM service not responding. Make sure the container is running.', 'error');
                    }
                } catch (error) {
                    this.showStatus('❌ Cannot connect to NIM service. Check if the container is running on port 8000.', 'error');
                }
            }
            
            async generateModel() {
                const prompt = this.promptInput.value.trim();
                const seed = parseInt(this.seedInput.value) || 0;
                
                if (!prompt) {
                    this.showStatus('❌ Please enter a prompt', 'error');
                    return;
                }
                
                this.setLoading(true);
                this.showStatus(`🚀 Generating 3D model: "${prompt}"...`, 'info');
                
                try {
                    const blob = await this.nimService.generateModel(prompt, seed);
                    const modelUrl = this.nimService.createModelUrl(blob);
                    
                    this.displayModel(modelUrl);
                    this.downloadModel(blob, `${prompt.replace(/[^a-zA-Z0-9]/g, '_')}.glb`);
                    
                    this.showStatus(`✅ 3D model generated successfully! (${(blob.size / 1024).toFixed(1)} KB)`, 'success');
                    
                } catch (error) {
                    console.error('Generation error:', error);
                    this.showStatus(`❌ Generation failed: ${error.message}`, 'error');
                } finally {
                    this.setLoading(false);
                }
            }
            
            displayModel(modelUrl) {
                this.modelViewer.src = modelUrl;
                this.modelViewer.classList.remove('hidden');
                this.viewerPlaceholder.classList.add('hidden');
                
                this.modelViewer.addEventListener('error', (e) => {
                    console.error('Model loading error:', e);
                    this.showStatus('❌ Error loading 3D model', 'error');
                });
                
                this.modelViewer.addEventListener('load', () => {
                    console.log('✅ 3D model loaded successfully');
                });
            }
            
            downloadModel(blob, filename) {
                const link = document.createElement('a');
                link.href = URL.createObjectURL(blob);
                link.download = filename;
                document.body.appendChild(link);
                link.click();
                document.body.removeChild(link);
                URL.revokeObjectURL(link.href);
            }
            
            setLoading(loading) {
                this.generateBtn.disabled = loading;
                if (loading) {
                    this.generateBtn.textContent = '⏳ Generating...';
                    this.loadingSpinner.style.display = 'block';
                } else {
                    this.generateBtn.textContent = '🚀 Generate 3D Model';
                    this.loadingSpinner.style.display = 'none';
                }
            }
            
            showStatus(message, type) {
                this.statusDiv.textContent = message;
                this.statusDiv.className = `status ${type}`;
                this.statusDiv.classList.remove('hidden');
                
                if (type === 'success') {
                    setTimeout(() => {
                        this.statusDiv.classList.add('hidden');
                    }, 5000);
                }
            }
        }

        // Initialize the app when DOM is loaded
        document.addEventListener('DOMContentLoaded', () => {
            console.log('🎨 Initializing Trellis 3D Generator...');
            window.trellisApp = new TrellisApp();
        });
    </script>
</body>
</html>'''

# Write the HTML file
with open('index.html', 'w') as f:
    f.write(html_content)

print('✅ Complete frontend created: index.html')
print('   📄 Beautiful modern UI with 3D viewer')
print('   🔧 Embedded NIM service integration')
print('   ⚙️ Full application logic included')

# Start server with CORS support
class CORSHTTPRequestHandler(SimpleHTTPRequestHandler):
    def end_headers(self):
        self.send_header('Access-Control-Allow-Origin', '*')
        self.send_header('Access-Control-Allow-Methods', 'GET, POST, OPTIONS')
        self.send_header('Access-Control-Allow-Headers', 'Content-Type')
        super().end_headers()
    
    def do_OPTIONS(self):
        self.send_response(200)
        self.end_headers()

def start_server(port=3000):
    try:
        with socketserver.TCPServer(("", port), CORSHTTPRequestHandler) as httpd:
            print(f"🌐 Frontend server started on http://localhost:{port}")
            httpd.serve_forever()
    except Exception as e:
        print(f"❌ Failed to start server: {e}")

# Launch server in background
server_thread = threading.Thread(target=start_server, args=(3000,))
server_thread.daemon = True
server_thread.start()

time.sleep(2)

# Open browser
try:
    webbrowser.open('http://localhost:3000')
    print("🌐 Opening browser automatically...")
except:
    pass

print("\n" + "="*60)
print("🎉 TRELLIS 3D GENERATOR LAUNCHED!")
print("="*60)
print("🔗 URL: http://localhost:3000")
print("🔧 NIM Service: http://localhost:8000")
print("\n✨ Features:")
print("   • 🎨 Beautiful modern UI with 3D viewer")
print("   • 🎯 Real-time model generation")
print("   • 📱 Mobile responsive design")
print("   • 💾 Automatic .glb file downloads")
print("   • 🔄 Interactive 3D model viewer")
print("   • 💡 Example prompts included")
print("\n🚀 Ready to create amazing 3D models!")

# Return to original directory
os.chdir('..')


## Step 4: Launch Frontend UI

Now let's set up and launch the integrated frontend UI that will provide a web interface for interacting with your Trellis NIM service.


In [ ]:
# 🚀 ONE-CLICK FRONTEND UI LAUNCHER
# This cell creates and launches a complete 3D model generation interface

import os
import json
import threading
import time
import webbrowser
from http.server import HTTPServer, SimpleHTTPRequestHandler
import socketserver

print("🎨 Creating Complete Frontend UI...")

# Create frontend directory
os.makedirs('trellis-frontend', exist_ok=True)
os.chdir('trellis-frontend')

# Create the main HTML file with embedded 3D viewer
html_content = '''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Trellis 3D Generator</title>
    <script type="module" src="https://unpkg.com/@google/model-viewer/dist/model-viewer.min.js"></script>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }

        body {
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            display: flex;
        }

        .container {
            display: flex;
            width: 100%;
            max-width: 1400px;
            margin: 0 auto;
            background: white;
            border-radius: 20px;
            box-shadow: 0 20px 40px rgba(0, 0, 0, 0.1);
            overflow: hidden;
            margin: 20px;
        }

        .left-panel {
            flex: 1;
            padding: 40px;
            background: #f8fafc;
        }

        .right-panel {
            flex: 1.5;
            background: #1a1a2e;
            position: relative;
            display: flex;
            flex-direction: column;
        }

        h1 {
            color: #2d3748;
            font-size: 2.5rem;
            font-weight: 700;
            margin-bottom: 10px;
        }

        .subtitle {
            color: #718096;
            font-size: 1.1rem;
            margin-bottom: 30px;
        }

        .form-group {
            margin-bottom: 25px;
        }

        label {
            display: block;
            font-weight: 600;
            color: #4a5568;
            margin-bottom: 8px;
            font-size: 0.95rem;
        }

        #promptInput {
            width: 100%;
            padding: 15px 20px;
            font-size: 16px;
            border: 2px solid #e2e8f0;
            border-radius: 12px;
            background: white;
            transition: all 0.3s ease;
            font-family: inherit;
        }

        #promptInput:focus {
            outline: none;
            border-color: #667eea;
            box-shadow: 0 0 0 3px rgba(102, 126, 234, 0.1);
        }

        .controls {
            display: flex;
            gap: 15px;
            margin-bottom: 25px;
        }

        .seed-input {
            flex: 1;
        }

        #seedInput {
            width: 100%;
            padding: 12px 15px;
            border: 2px solid #e2e8f0;
            border-radius: 8px;
            font-size: 14px;
        }

        #generateBtn {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            border: none;
            padding: 15px 30px;
            font-size: 16px;
            font-weight: 600;
            border-radius: 12px;
            cursor: pointer;
            transition: all 0.3s ease;
            box-shadow: 0 4px 15px rgba(102, 126, 234, 0.3);
        }

        #generateBtn:hover:not(:disabled) {
            transform: translateY(-2px);
            box-shadow: 0 6px 20px rgba(102, 126, 234, 0.4);
        }

        #generateBtn:disabled {
            opacity: 0.6;
            cursor: not-allowed;
            transform: none;
        }

        .status {
            padding: 15px 20px;
            border-radius: 8px;
            margin-bottom: 20px;
            font-weight: 500;
        }

        .status.info {
            background: #ebf8ff;
            color: #2b6cb0;
            border: 1px solid #90cdf4;
        }

        .status.success {
            background: #f0fff4;
            color: #276749;
            border: 1px solid #9ae6b4;
        }

        .status.error {
            background: #fed7d7;
            color: #c53030;
            border: 1px solid #feb2b2;
        }

        .examples {
            background: white;
            padding: 20px;
            border-radius: 12px;
            border: 1px solid #e2e8f0;
        }

        .examples h3 {
            color: #2d3748;
            margin-bottom: 15px;
            font-size: 1.1rem;
        }

        .example-item {
            padding: 8px 12px;
            background: #f7fafc;
            border-radius: 6px;
            cursor: pointer;
            transition: all 0.2s ease;
            margin-bottom: 8px;
            font-size: 0.9rem;
            color: #4a5568;
        }

        .example-item:hover {
            background: #edf2f7;
            color: #2d3748;
        }

        model-viewer {
            width: 100%;
            height: 60%;
            background: linear-gradient(135deg, #1a1a2e 0%, #16213e 50%, #0f3460 100%);
        }

        .viewer-info {
            padding: 30px;
            color: white;
            flex: 1;
            display: flex;
            flex-direction: column;
            justify-content: center;
            align-items: center;
            text-align: center;
        }

        .viewer-placeholder {
            font-size: 1.2rem;
            color: #cbd5e0;
            margin-bottom: 15px;
        }

        .viewer-instructions {
            color: #a0aec0;
            font-size: 0.9rem;
            max-width: 300px;
            line-height: 1.6;
        }

        .loading-spinner {
            display: none;
            width: 40px;
            height: 40px;
            border: 4px solid #f3f3f3;
            border-top: 4px solid #667eea;
            border-radius: 50%;
            animation: spin 1s linear infinite;
            margin: 20px auto;
        }

        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }

        .hidden {
            display: none !important;
        }

        @media (max-width: 768px) {
            .container {
                flex-direction: column;
                margin: 10px;
            }
            
            .left-panel, .right-panel {
                flex: none;
            }
            
            .right-panel {
                min-height: 400px;
            }
            
            h1 {
                font-size: 2rem;
            }
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="left-panel">
            <h1>🎨 Trellis 3D</h1>
            <p class="subtitle">Generate 3D models from text prompts using NVIDIA Trellis NIM</p>
            
            <div class="form-group">
                <label for="promptInput">Enter your prompt:</label>
                <input type="text" id="promptInput" 
                       placeholder="A modern chair with sleek design..." 
                       value="A modern wooden chair">
            </div>
            
            <div class="controls">
                <div class="seed-input">
                    <label for="seedInput">Seed (0 = random):</label>
                    <input type="number" id="seedInput" value="0" min="0">
                </div>
            </div>
            
            <button id="generateBtn">🚀 Generate 3D Model</button>
            <div class="loading-spinner" id="loadingSpinner"></div>
            
            <div id="status" class="status hidden"></div>
            
            <div class="examples">
                <h3>💡 Example Prompts:</h3>
                <div class="example-item" data-prompt="A modern wooden chair">A modern wooden chair</div>
                <div class="example-item" data-prompt="A futuristic sports car">A futuristic sports car</div>
                <div class="example-item" data-prompt="A cozy living room">A cozy living room</div>
                <div class="example-item" data-prompt="A medieval castle tower">A medieval castle tower</div>
                <div class="example-item" data-prompt="A space station module">A space station module</div>
                <div class="example-item" data-prompt="A vintage coffee shop interior">A vintage coffee shop interior</div>
            </div>
        </div>
        
        <div class="right-panel">
            <model-viewer id="modelViewer" class="hidden"
                         camera-controls 
                         enable-pan
                         auto-rotate
                         shadow-intensity="1"
                         environment-image="neutral">
            </model-viewer>
            
            <div class="viewer-info" id="viewerPlaceholder">
                <div class="viewer-placeholder">🎯 3D Model Viewer</div>
                <div class="viewer-instructions">
                    Enter a prompt and click "Generate 3D Model" to see your creation appear here. 
                    You'll be able to rotate, zoom, and interact with the generated 3D model.
                </div>
            </div>
        </div>
    </div>

    <script src="nim-service.js"></script>
    <script src="app.js"></script>
</body>
</html>'''

# Create the NIM service JavaScript
nim_service_js = '''
// NIM Service Integration
class NIMService {
    constructor() {
        this.config = {
            nim_endpoint: "http://localhost:8000",
            infer_endpoint: "http://localhost:8000/v1/infer",
            health_endpoint: "http://localhost:8000/health"
        };
        this.currentModelUrl = null;
    }
    
    async generateModel(prompt, seed = 0) {
        const payload = {
            prompt: prompt,
            seed: seed
        };
        
        try {
            console.log('🚀 Sending request to NIM:', payload);
            const response = await fetch(this.config.infer_endpoint, {
                method: 'POST',
                headers: {
                    'Accept': 'application/json',
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify(payload)
            });
            
            if (!response.ok) {
                throw new Error(`HTTP error! status: ${response.status}`);
            }
            
            const data = await response.json();
            console.log('✅ NIM response received:', data);
            
            if (data.artifacts && data.artifacts.length > 0) {
                const artifact = data.artifacts[0];
                if (artifact.base64) {
                    return this.base64ToBlob(artifact.base64);
                }
            }
            
            throw new Error('No valid artifact in response');
            
        } catch (error) {
            console.error('❌ NIM generation error:', error);
            throw error;
        }
    }
    
    base64ToBlob(base64Data) {
        try {
            const byteCharacters = atob(base64Data);
            const byteNumbers = new Array(byteCharacters.length);
            for (let i = 0; i < byteCharacters.length; i++) {
                byteNumbers[i] = byteCharacters.charCodeAt(i);
            }
            const byteArray = new Uint8Array(byteNumbers);
            return new Blob([byteArray], { type: 'model/gltf-binary' });
        } catch (error) {
            console.error('❌ Base64 decode error:', error);
            throw new Error('Failed to decode model data');
        }
    }
    
    createModelUrl(blob) {
        // Clean up previous URL to prevent memory leaks
        if (this.currentModelUrl) {
            URL.revokeObjectURL(this.currentModelUrl);
        }
        
        this.currentModelUrl = URL.createObjectURL(blob);
        return this.currentModelUrl;
    }
    
    async checkHealth() {
        try {
            const response = await fetch(this.config.health_endpoint, { timeout: 5000 });
            return response.ok;
        } catch (error) {
            try {
                const response = await fetch(this.config.nim_endpoint, { timeout: 5000 });
                return response.ok;
            } catch (fallbackError) {
                return false;
            }
        }
    }
}

// Make NIMService available globally
window.NIMService = NIMService;
'''

# Create the main application JavaScript
app_js = '''
// Main Application Logic
class TrellisApp {
    constructor() {
        this.nimService = new NIMService();
        this.initializeElements();
        this.attachEventListeners();
        this.checkNIMStatus();
    }
    
    initializeElements() {
        this.promptInput = document.getElementById('promptInput');
        this.seedInput = document.getElementById('seedInput');
        this.generateBtn = document.getElementById('generateBtn');
        this.loadingSpinner = document.getElementById('loadingSpinner');
        this.statusDiv = document.getElementById('status');
        this.modelViewer = document.getElementById('modelViewer');
        this.viewerPlaceholder = document.getElementById('viewerPlaceholder');
    }
    
    attachEventListeners() {
        this.generateBtn.addEventListener('click', () => this.generateModel());
        
        this.promptInput.addEventListener('keypress', (e) => {
            if (e.key === 'Enter' && !this.generateBtn.disabled) {
                this.generateModel();
            }
        });
        
        // Example prompt clicks
        document.querySelectorAll('.example-item').forEach(item => {
            item.addEventListener('click', () => {
                this.promptInput.value = item.dataset.prompt;
                this.promptInput.focus();
            });
        });
    }
    
    async checkNIMStatus() {
        try {
            const isHealthy = await this.nimService.checkHealth();
            if (isHealthy) {
                this.showStatus('✅ Connected to Trellis NIM service', 'success');
            } else {
                this.showStatus('⚠️ NIM service not responding. Make sure the container is running.', 'error');
            }
        } catch (error) {
            this.showStatus('❌ Cannot connect to NIM service. Check if the container is running on port 8000.', 'error');
        }
    }
    
    async generateModel() {
        const prompt = this.promptInput.value.trim();
        const seed = parseInt(this.seedInput.value) || 0;
        
        if (!prompt) {
            this.showStatus('❌ Please enter a prompt', 'error');
            return;
        }
        
        this.setLoading(true);
        this.showStatus(`🚀 Generating 3D model: "${prompt}"...`, 'info');
        
        try {
            const blob = await this.nimService.generateModel(prompt, seed);
            const modelUrl = this.nimService.createModelUrl(blob);
            
            // Show the 3D model
            this.displayModel(modelUrl);
            
            // Save the model file
            this.downloadModel(blob, `${prompt.replace(/[^a-zA-Z0-9]/g, '_')}.glb`);
            
            this.showStatus(`✅ 3D model generated successfully! (${(blob.size / 1024).toFixed(1)} KB)`, 'success');
            
        } catch (error) {
            console.error('Generation error:', error);
            this.showStatus(`❌ Generation failed: ${error.message}`, 'error');
        } finally {
            this.setLoading(false);
        }
    }
    
    displayModel(modelUrl) {
        this.modelViewer.src = modelUrl;
        this.modelViewer.classList.remove('hidden');
        this.viewerPlaceholder.classList.add('hidden');
        
        // Add error handling for model loading
        this.modelViewer.addEventListener('error', (e) => {
            console.error('Model loading error:', e);
            this.showStatus('❌ Error loading 3D model', 'error');
        });
        
        this.modelViewer.addEventListener('load', () => {
            console.log('✅ 3D model loaded successfully');
        });
    }
    
    downloadModel(blob, filename) {
        const link = document.createElement('a');
        link.href = URL.createObjectURL(blob);
        link.download = filename;
        document.body.appendChild(link);
        link.click();
        document.body.removeChild(link);
        URL.revokeObjectURL(link.href);
    }
    
    setLoading(loading) {
        this.generateBtn.disabled = loading;
        if (loading) {
            this.generateBtn.textContent = '⏳ Generating...';
            this.loadingSpinner.style.display = 'block';
        } else {
            this.generateBtn.textContent = '🚀 Generate 3D Model';
            this.loadingSpinner.style.display = 'none';
        }
    }
    
    showStatus(message, type) {
        this.statusDiv.textContent = message;
        this.statusDiv.className = `status ${type}`;
        this.statusDiv.classList.remove('hidden');
        
        // Auto-hide success messages after 5 seconds
        if (type === 'success') {
            setTimeout(() => {
                this.statusDiv.classList.add('hidden');
            }, 5000);
        }
    }
}

// Initialize the app when DOM is loaded
document.addEventListener('DOMContentLoaded', () => {
    console.log('🎨 Initializing Trellis 3D Generator...');
    window.trellisApp = new TrellisApp();
});
'''

# Write all files
with open('index.html', 'w') as f:
    f.write(html_content)

with open('nim-service.js', 'w') as f:
    f.write(nim_service_js)

with open('app.js', 'w') as f:
    f.write(app_js)

print('✅ Frontend files created:')
print('   📄 index.html - Main UI with 3D viewer')
print('   🔧 nim-service.js - NIM API integration')
print('   ⚙️ app.js - Application logic')

# Start the server with CORS support
class CORSHTTPRequestHandler(SimpleHTTPRequestHandler):
    def end_headers(self):
        self.send_header('Access-Control-Allow-Origin', '*')
        self.send_header('Access-Control-Allow-Methods', 'GET, POST, OPTIONS')
        self.send_header('Access-Control-Allow-Headers', 'Content-Type')
        super().end_headers()
    
    def do_OPTIONS(self):
        self.send_response(200)
        self.end_headers()

def start_server(port=3000):
    try:
        with socketserver.TCPServer(("", port), CORSHTTPRequestHandler) as httpd:
            print(f"🌐 Frontend server started on http://localhost:{port}")
            httpd.serve_forever()
    except Exception as e:
        print(f"❌ Failed to start server: {e}")

# Launch server in background thread
server_thread = threading.Thread(target=start_server, args=(3000,))
server_thread.daemon = True
server_thread.start()

# Give server time to start
time.sleep(2)

# Try to open browser
try:
    webbrowser.open('http://localhost:3000')
    print("🌐 Opening browser automatically...")
except:
    pass

print("\n" + "="*60)
print("🎉 TRELLIS 3D GENERATOR LAUNCHED!")
print("="*60)
print("🔗 URL: http://localhost:3000")
print("🔧 NIM Service: http://localhost:8000")
print("\n✨ Features:")
print("   • 🎨 Beautiful modern UI with 3D viewer")
print("   • 🎯 Real-time model generation")
print("   • 📱 Mobile responsive design")
print("   • 💾 Automatic .glb file downloads")
print("   • 🔄 Interactive 3D model viewer")
print("   • 💡 Example prompts included")
print("\n🚀 Ready to create amazing 3D models!")

# Return to original directory
os.chdir('..')


In [ ]:
# Examine the frontend structure and setup
import json
import shutil

print('🔍 Examining frontend structure...')
!find prompt-to-trellis -type f -name "*.html" -o -name "*.js" -o -name "*.css" -o -name "*.json" | head -10

# Check if it's a Node.js/npm project
if os.path.exists('prompt-to-trellis/package.json'):
    print('📦 Found package.json - This is a Node.js project')
    with open('prompt-to-trellis/package.json', 'r') as f:
        package_data = json.load(f)
    print(f'Project: {package_data.get("name", "Unknown")}')
    print(f'Description: {package_data.get("description", "No description")}')
    
    # Install dependencies if needed
    print('📥 Installing frontend dependencies...')
    !cd prompt-to-trellis && npm install
else:
    print('📄 Appears to be a static HTML/JS project')

# Check for main files
main_files = ['index.html', 'main.js', 'app.js', 'script.js']
found_files = []
for file in main_files:
    if os.path.exists(f'prompt-to-trellis/{file}'):
        found_files.append(file)

print(f'📋 Main files found: {found_files}')


In [ ]:
# Create API bridge configuration for the frontend
import json

# Create a configuration file that the frontend can use to connect to NIM
config = {
    "nim_endpoint": "http://localhost:8000",
    "api_base": "http://localhost:8000/v1",
    "infer_endpoint": "http://localhost:8000/v1/infer",
    "health_endpoint": "http://localhost:8000/health"
}

# Write config to the frontend directory
with open('prompt-to-trellis/nim-config.json', 'w') as f:
    json.dump(config, f, indent=2)

print('⚙️ Created NIM service configuration for frontend')

# Create a simple JavaScript module for NIM integration
nim_integration_js = '''
// NIM Service Integration
class NIMService {
    constructor() {
        this.config = null;
        this.loadConfig();
    }
    
    async loadConfig() {
        try {
            const response = await fetch('./nim-config.json');
            this.config = await response.json();
            console.log('NIM config loaded:', this.config);
        } catch (error) {
            console.error('Failed to load NIM config:', error);
            // Fallback config
            this.config = {
                nim_endpoint: "http://localhost:8000",
                infer_endpoint: "http://localhost:8000/v1/infer"
            };
        }
    }
    
    async generateModel(prompt, seed = 0) {
        if (!this.config) {
            await this.loadConfig();
        }
        
        const payload = {
            prompt: prompt,
            seed: seed
        };
        
        try {
            console.log('Sending request to NIM:', payload);
            const response = await fetch(this.config.infer_endpoint, {
                method: 'POST',
                headers: {
                    'Accept': 'application/json',
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify(payload)
            });
            
            if (!response.ok) {
                throw new Error(`HTTP error! status: ${response.status}`);
            }
            
            const data = await response.json();
            console.log('NIM response:', data);
            
            if (data.artifacts && data.artifacts.length > 0) {
                const artifact = data.artifacts[0];
                if (artifact.base64) {
                    return this.base64ToBlob(artifact.base64);
                }
            }
            
            throw new Error('No valid artifact in response');
            
        } catch (error) {
            console.error('NIM generation error:', error);
            throw error;
        }
    }
    
    base64ToBlob(base64Data) {
        const byteCharacters = atob(base64Data);
        const byteNumbers = new Array(byteCharacters.length);
        for (let i = 0; i < byteCharacters.length; i++) {
            byteNumbers[i] = byteCharacters.charCodeAt(i);
        }
        const byteArray = new Uint8Array(byteNumbers);
        return new Blob([byteArray], { type: 'model/gltf-binary' });
    }
    
    async checkHealth() {
        if (!this.config) {
            await this.loadConfig();
        }
        
        try {
            const response = await fetch(this.config.health_endpoint || this.config.nim_endpoint + '/health');
            return response.ok;
        } catch (error) {
            return false;
        }
    }
}

// Make NIMService available globally
window.NIMService = NIMService;
'''

# Write the NIM integration JavaScript
with open('prompt-to-trellis/nim-service.js', 'w') as f:
    f.write(nim_integration_js)

print('✅ Created NIM service JavaScript integration module')
print('📂 Files created:')
print('   - nim-config.json (configuration)')
print('   - nim-service.js (JavaScript integration)')

# Show the user what to do next
print('\\n🔧 To integrate with your frontend:')
print('1. Include nim-service.js in your HTML: <script src="nim-service.js"></script>')
print('2. Create a NIMService instance: const nim = new NIMService();')
print('3. Use nim.generateModel(prompt) to generate 3D models')
print('4. The returned blob can be used as a URL for 3D viewers')


In [ ]:
# Launch the frontend UI
import threading
import time
import webbrowser
from http.server import HTTPServer, SimpleHTTPRequestHandler
import socketserver
import os

def start_frontend_server(port=3000):
    """Start the frontend server in a separate thread"""
    class CORSHTTPRequestHandler(SimpleHTTPRequestHandler):
        def end_headers(self):
            self.send_header('Access-Control-Allow-Origin', '*')
            self.send_header('Access-Control-Allow-Methods', 'GET, POST, OPTIONS')
            self.send_header('Access-Control-Allow-Headers', 'Content-Type')
            super().end_headers()
        
        def do_OPTIONS(self):
            self.send_response(200)
            self.end_headers()
    
    os.chdir('prompt-to-trellis')
    
    try:
        with socketserver.TCPServer(("", port), CORSHTTPRequestHandler) as httpd:
            print(f"🌐 Frontend server started on http://localhost:{port}")
            httpd.serve_forever()
    except Exception as e:
        print(f"❌ Failed to start server on port {port}: {e}")

# Start the server in background
def launch_frontend():
    frontend_port = 3000
    
    # Check if the frontend files exist
    if not os.path.exists('prompt-to-trellis'):
        print("❌ Frontend directory not found. Please run the clone cell first.")
        return
    
    print("🚀 Starting frontend server...")
    
    # Start server in a separate thread
    server_thread = threading.Thread(target=start_frontend_server, args=(frontend_port,))
    server_thread.daemon = True
    server_thread.start()
    
    # Give the server a moment to start
    time.sleep(2)
    
    print(f"✅ Frontend UI is now available at: http://localhost:{frontend_port}")
    print("🎯 The UI is connected to your NIM service running on port 8000")
    print("💡 Use the web interface to generate 3D models with prompts")
    
    # Try to open the browser automatically
    try:
        webbrowser.open(f'http://localhost:{frontend_port}')
        print("🌐 Opening browser automatically...")
    except:
        print("🌐 Please manually open http://localhost:3000 in your browser")
    
    return f"http://localhost:{frontend_port}"

# Launch the frontend
frontend_url = launch_frontend()

# Display instructions
print("\\n" + "="*60)
print("🎉 FRONTEND UI LAUNCHED!")
print("="*60)
print(f"🔗 URL: {frontend_url}")
print("🔧 NIM Service: http://localhost:8000")
print("\\n📝 How to use:")
print("1. Open the URL above in your browser")
print("2. Enter a prompt in the text field")
print("3. Click generate to create a 3D model")
print("4. The .glb model will appear in the viewer on the right")
print("\\n💡 Example prompts:")
print("   • 'A modern chair'")
print("   • 'A wooden table'") 
print("   • 'A cozy bedroom'")
print("   • 'A futuristic car'")


In [ ]:
# Alternative: Launch with Node.js/npm if available
import subprocess

def launch_with_npm():
    """Launch frontend using npm if package.json exists"""
    if os.path.exists('prompt-to-trellis/package.json'):
        print("📦 Detected npm project, attempting to launch with npm...")
        try:
            os.chdir('prompt-to-trellis')
            
            # Try common npm start commands
            for cmd in ['npm start', 'npm run dev', 'npm run serve']:
                try:
                    print(f"🚀 Trying: {cmd}")
                    process = subprocess.Popen(cmd.split(), 
                                             stdout=subprocess.PIPE, 
                                             stderr=subprocess.PIPE)
                    time.sleep(3)  # Give it time to start
                    
                    if process.poll() is None:  # Process is still running
                        print(f"✅ Successfully started with: {cmd}")
                        print("🌐 Check the output above for the URL (usually http://localhost:3000)")
                        return True
                    
                except Exception as e:
                    print(f"⚠️ {cmd} failed: {e}")
                    continue
            
            print("ℹ️ npm commands didn't work, falling back to Python server")
            return False
            
        except Exception as e:
            print(f"❌ npm launch failed: {e}")
            return False
    return False

# Try npm first, fall back to Python server
if not launch_with_npm():
    # The Python server is already launched above
    pass


## Management Commands

Use these cells to manage your NIM container:


In [ ]:
# Check NIM container status
import subprocess

print("🔍 NIM STATUS CHECK")
print("="*40)

# Check NIM container
print("📦 NIM Container Status:")
nim_check = subprocess.run(['docker', 'ps'], capture_output=True, text=True)
if 'nim-server' in nim_check.stdout:
    print("✅ NIM container is running")
    !docker ps | grep nim-server
else:
    print("❌ NIM container is not running")

# Check NIM API health  
print("\n🔗 NIM API Health:")
import requests
try:
    response = requests.get("http://localhost:8000/health", timeout=5)
    if response.status_code == 200:
        print("✅ NIM API is responding")
    else:
        print(f"⚠️ NIM API returned status: {response.status_code}")
except:
    try:
        response = requests.get("http://localhost:8000/", timeout=5)
        print("✅ NIM service is responding")
    except:
        print("❌ NIM API is not accessible")

print("\n📊 Port 8000 Status:")
!netstat -an | grep ":8000" | grep LISTEN || echo "No service found on port 8000"


In [ ]:
# View recent container logs
!docker logs nim-server --tail 20


In [ ]:
# Stop the NIM container (when you're done)
!docker stop nim-server
!docker rm nim-server
print("🛑 Trellis NIM container stopped and removed")


## 🎉 Generation Complete!

### To generate more 3D models:
1. Go back to the cell above and change `PROMPT = "..."`
2. Optionally change `OUTPUT_FILE = "..."` to save with a different name
3. Run the cell again

### Example prompts to try:
- `"A modern chair"`
- `"A futuristic car"`
- `"A wooden table"`
- `"A cozy bedroom"`
- `"A space station interior"`
- `"A medieval castle tower"`
- `"A vintage coffee shop interior"`


In [ ]:
# View recent container logs
!docker logs nim-server --tail 20


In [ ]:
# Stop the NIM container (when you're done)
!docker stop nim-server
!docker rm nim-server
print("🛑 Trellis NIM container stopped and removed")


In [ ]:
# List generated GLB files
!ls -la *.glb


## 🎯 Complete!

### **🎉 Your Trellis NIM setup is now complete!**

**What you have:**
- ✅ **NVIDIA Trellis NIM** deployed and running
- ✅ **Command-line generation** - Simple, reliable 3D model creation
- ✅ **GLB output files** - Ready for Blender, Unity, or any 3D software
- ✅ **Container management** - Easy start, stop, and monitoring

### **🚀 Workflow:**
1. **Deploy NIM**: Run cells 2-8 (one-time setup)
2. **Generate models**: Use cell 10, change the prompt and run
3. **Manage container**: Use cells 12-15 for monitoring and cleanup

### **🛠️ Troubleshooting:**
- **Container not responding** → Check `docker logs nim-server`
- **Generation fails** → Verify container is running with `docker ps | grep nim-server`
- **Out of memory** → Restart container with `docker restart nim-server`

### **📁 Files:**
- **Generated Models**: `.glb` files in the current directory
- **NIM Service**: Running on http://localhost:8000

**🎨 Ready to create amazing 3D models from text prompts!**
